<a href="https://colab.research.google.com/github/NotAndex/Demo/blob/main/time_series_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0

# 1 Setup


In [14]:
!pip install yfinance
import yfinance as yf

import numpy as np
import pandas as pd
import datetime

from tensorflow.keras.utils import to_categorical

# 2 Data

In [ ]:
today = datetime.date.today()
start_date = today - datetime.timedelta(days=365*3)
symbol = 'BTC-USD'

cc_data = yf.download(symbol, start=start_date, end=today)
cc_data.reset_index(inplace=True,col_fill=('Date'))


#3 Feature engineering examples

## 3.1 Date dependent features

In [ ]:
cc_data['year'] = cc_data['Date'].dt.year 
cc_data['quarter'] = cc_data['Date'].dt.quarter 
cc_data['month'] = cc_data['Date'].dt.month 
cc_data['week'] = cc_data['Date'].dt.isocalendar().week 
cc_data['day'] = cc_data['Date'].dt.day
cc_data['wday'] = cc_data['Date'].dt.dayofweek

cc_data

In [17]:
monthCat = pd.DataFrame(to_categorical(cc_data.loc[:,['month']]))
cc_data = pd.concat([cc_data,monthCat.loc[:,1:]],axis=1)

cc_data

,Date,Open,High,Low,Close,Adj Close,Volume,year,quarter,month,week,day,wday,1,2,3,4,5,6,7,8,9,10,11,12,1,2,3,4,5,6,7,8,9,10,11,12,1,2,3,4,5,6,7,8,9,10,11,12
0,2018-06-03,7632.089844,7754.890137,7613.040039,7720.250000,7720.250000,4851760128,2018,2,6,22,3,6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-06-04,7722.529785,7753.819824,7474.040039,7514.470215,7514.470215,4993169920,2018,2,6,23,4,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-06-05,7500.899902,7643.229980,7397.000000,7633.759766,7633.759766,4961739776,2018,2,6,23,5,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-06-06,7625.970215,7680.430176,7502.009766,7653.979980,7653.979980,4692259840,2018,2,6,23,6,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-06-07,7650.819824,7741.270020,7650.819824,7678.240234,7678.240234,4485799936,2018,2,6,23,7,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,2021-05-29,35684.156250,37234.500000,33693.929688,34616.066406,34616.066406,45231013335,2021,2,5,21,29,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1088,2021-05-30,34607.406250,36400.667969,33520.738281,35678.128906,35678.128906,31646080921,2021,2,5,21,30,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1089,2021-05-31,35658.593750,37468.250000,34241.945312,37332.855469,37332.855469,39009847639,2021,2,5,22,31,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1090,2021-06-01,37293.792969,37896.734375,35787.085938,36684.925781,36684.925781,34639423297,2021,2,6,22,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# 3.2 Technical features (Indicators **$f_{x}$**)

# 3.3 External Features
e.g Sentinment